# Jigsaw Puzzle — Milestone 1 Pipeline

This notebook implements a full Milestone-1 pipeline for the puzzle dataset located at `/mnt/data/1.jpg`.

**Features implemented:**
- Automatic grid detection (projection profiles) with fallback to manual grid size
- Tile extraction for 2x2, 4x4, 8x8 puzzles
- Per-tile enhancement (denoise, CLAHE, optional resize)
- Binary mask creation and contour extraction
- Artifacts saving (enhanced images, masks, contours, metadata)
- Visualization helpers (before/after and grid overlay)

Run each cell in order. Modify parameters near the top if needed.


In [ ]:
import cv2, os, json, numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
# ============================  
# USER SETTINGS
# ============================
IMAGE_PATH = "D:/Image-Processing-Project/Gravity_Falls/puzzle_4x4/15.jpg"
# Optional: set GRID_SIZE manually or leave None to infer automatically
GRID_SIZE = None

# ============================
# OUTPUT FOLDERS
# ============================
BASE_OUTPUT = "D:/Image-Processing-Project/output"
TILES_DIR = os.path.join(BASE_OUTPUT, "tiles")
CONTOURS_DIR = os.path.join(BASE_OUTPUT, "contours")
EDGES_DIR = os.path.join(BASE_OUTPUT, "edges")
VIZ_DIR = os.path.join(BASE_OUTPUT, "visualizations")

os.makedirs(TILES_DIR, exist_ok=True)
os.makedirs(CONTOURS_DIR, exist_ok=True)
os.makedirs(EDGES_DIR, exist_ok=True)
os.makedirs(VIZ_DIR, exist_ok=True)

# ============================
# LOAD IMAGE
# ============================
img = cv2.imread(IMAGE_PATH)
if img is None:
    raise Exception("❌ Image not found! Check IMAGE_PATH.")

h, w, _ = img.shape

# ============================
# INFER GRID SIZE (if not specified)
# ============================
if GRID_SIZE is None:
    # Try to guess from folder name: look for 2x2, 4x4, 8x8
    folder_name = os.path.basename(os.path.dirname(IMAGE_PATH))
    if "2x2" in folder_name:
        GRID_SIZE = 2
    elif "4x4" in folder_name:
        GRID_SIZE = 4
    elif "8x8" in folder_name:
        GRID_SIZE = 8
    else:
        # Fallback: assume smallest 2x2
        GRID_SIZE = 2
print(f"Using GRID_SIZE = {GRID_SIZE}")

tile_h = h // GRID_SIZE
tile_w = w // GRID_SIZE

tile_count = 0
full_contour_img = img.copy()

# ============================
# GRID SEGMENTATION + TILE EXTRACTION
# ============================
for row in range(GRID_SIZE):
    for col in range(GRID_SIZE):
        y1, y2 = row * tile_h, (row + 1) * tile_h
        x1, x2 = col * tile_w, (col + 1) * tile_w

        tile = img[y1:y2, x1:x2]

        # Save tile
        tile_filename = f"tile_{tile_count}.png"
        cv2.imwrite(os.path.join(TILES_DIR, tile_filename), tile)

        # Edge extraction
        gray = cv2.cvtColor(tile, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blur, 50, 150)
        edge_filename = f"edges_{tile_count}.png"
        cv2.imwrite(os.path.join(EDGES_DIR, edge_filename), edges)

        # Contour extraction
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contour_img = tile.copy()
        cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)
        contour_filename = f"contour_{tile_count}.png"
        cv2.imwrite(os.path.join(CONTOURS_DIR, contour_filename), contour_img)

        # Draw contours on full image
        for cnt in contours:
            cnt_offset = cnt + [x1, y1]
            cv2.drawContours(full_contour_img, [cnt_offset], -1, (0, 255, 0), 2)

        tile_count += 1

# ============================
# GRID VISUALIZATION IMAGE
# ============================
grid_viz = img.copy()
for i in range(1, GRID_SIZE):
    cv2.line(grid_viz, (0, i * tile_h), (w, i * tile_h), (0, 0, 255), 2)
    cv2.line(grid_viz, (i * tile_w, 0), (i * tile_w, h), (0, 0, 255), 2)

cv2.imwrite(os.path.join(VIZ_DIR, "grid_visualization.png"), grid_viz)

# Save original image with contours
cv2.imwrite(os.path.join(VIZ_DIR, "original_with_contours.png"), full_contour_img)

print("✅ Tile extraction completed.")
print("✅ Edge images saved in 'edges/' folder.")
print("✅ Contours extracted and saved in 'contours/' folder.")
print("✅ Original image with contours saved in 'visualizations/' folder.")
print("✅ Grid visualization saved in 'visualizations/' folder.")


Using GRID_SIZE = 4
✅ Tile extraction completed.
✅ Edge images saved in 'edges/' folder.
✅ Contours extracted and saved in 'contours/' folder.
✅ Original image with contours saved in 'visualizations/' folder.
✅ Grid visualization saved in 'visualizations/' folder.


In [ ]:
import cv2
import numpy as np
import os


def detect_salt_noise(img, bright_thresh=220, ratio=0.001):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    salt_ratio = np.sum(gray > bright_thresh) / gray.size
    return salt_ratio > ratio, salt_ratio


def detect_pepper_noise_median(img, dark_thresh=60, med_kernel=3, diff_threshold=30, ratio=0.001):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    median = cv2.medianBlur(gray, med_kernel)

    pepper_mask = np.logical_and(gray < dark_thresh, (median - gray) >= diff_threshold)
    ratio_median = np.sum(pepper_mask) / gray.size

    return (ratio_median > ratio, ratio_median)


def detect_gaussian_noise(img, threshold=15):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    noise_level = gray.std()
    return noise_level > threshold


def detect_blur(img, threshold=120):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    return fm < threshold


def enhance_image(img):
    print("\n===== IMAGE ANALYSIS =====")
    
    has_salt, salt_ratio = detect_salt_noise(img)
    has_pepper, pepper_ratio = detect_pepper_noise_median(img)
    has_gaussian = detect_gaussian_noise(img, threshold=50) 
    is_blurry = detect_blur(img, threshold=80)               

    print(f"Salt noise detected: {has_salt}  (ratio={salt_ratio:.5f})")
    print(f"Pepper noise detected: {has_pepper} (ratio={pepper_ratio:.5f})")
    print(f"Gaussian noise detected: {has_gaussian}")
    print(f"Image is blurry: {is_blurry}\n")


    if (
        not has_salt and salt_ratio < 0.0005 and
        not has_pepper and pepper_ratio < 0.0005 and
        not has_gaussian and
        not is_blurry
    ):
        print("Image is clean. No enhancement applied.")
        return img  

    enhanced = img.copy()

    # Only apply sharpening if blur is significant
    if is_blurry:
        blurred = cv2.GaussianBlur(enhanced, (9, 9), 0)
        enhanced = cv2.addWeighted(enhanced, 1.2, blurred, -0.2, 0)
        print("Sharpening applied.")

    if has_salt or has_pepper:
        enhanced = cv2.medianBlur(enhanced, 9)

    if has_gaussian:
        enhanced = cv2.bilateralFilter(enhanced, 9, 75, 75)

    return enhanced


if __name__ == "__main__":

    image_path = r"C:\Users\nada\Downloads\Image-Processing-Project\Gravity_Falls\nnn.png"
    save_path  = r"C:\Users\nada\Downloads\Image-Processing-Project\data_enhanced\puzzle1_enhanced.jpg"

    img = cv2.imread(image_path)

    if img is None:
        print("Error: Image not found")
    else:
        enhanced = enhance_image(img)

        scale = 0.4
        img_small = cv2.resize(img, None, fx=scale, fy=scale)
        enh_small = cv2.resize(enhanced, None, fx=scale, fy=scale)

        combined = np.hstack((img_small, enh_small))
        cv2.imshow("Original | Enhanced", combined)

        diff = cv2.absdiff(img_small, enh_small)
        cv2.imshow("Difference", diff)

        cv2.waitKey(0)
        cv2.destroyAllWindows()

        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        cv2.imwrite(save_path, enhanced)
        print(f"Enhanced image saved at: {save_path}")



===== IMAGE ANALYSIS =====
Salt noise detected: True  (ratio=0.02391)
Pepper noise detected: True (ratio=0.00745)
Gaussian noise detected: False
Image is blurry: False

Enhanced image saved at: C:\Users\nada\Downloads\Image-Processing-Project\data_enhanced\puzzle1_enhanced.jpg
